In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import os

# Load embeddings model (use OpenAI or HuggingFace embeddings)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Path where FAISS index is stored
faiss_index_path = "/data/user_data/vidhij2/medical_db/heirchical_chunking_rag"

# Load FAISS vector store
vector_store = FAISS.load_local(folder_path=faiss_index_path, embeddings=embeddings, allow_dangerous_deserialization=True)

print(f"FAISS index loaded successfully with {vector_store.index.ntotal} vvectors.")

/tmp/ipykernel_2725100/2819816299.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


FAISS index loaded successfully with 3020 vvectors.


In [9]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Example medical query
query = "What are the early signs of pregnancy complications?"

# Retrieve relevant document chunks
retrieved_chunks = retriever.get_relevant_documents(query)

# Print retrieved content
for i, chunk in enumerate(retrieved_chunks):
    print(f"Retrieved Chunk {i+1}: {chunk.page_content[:300]}...\n")

/tmp/ipykernel_3910775/3314463875.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_chunks = retriever.get_relevant_documents(query)


Retrieved Chunk 1: Pregnant now or within the last year?
Get medical care right away if you experience any of the following symptoms:
These could be signs of very serious complications. If you can’t reach a healthcare provider, go to the
emergency room. Be sure to tell them you are pregnant or were pregnant within the...

Retrieved Chunk 2: Normal symptoms during pregnancy – Nausea and vomiting, heartburn, constipation, and increased
frequency of urination. These symptoms may cause discomfort to the woman.
Examine symptoms of complications – Fever, persistent vomiting with dehydration, palpitations,
tiredness, breathlessness at rest/on...

Retrieved Chunk 3: medical facility when
labour starts or in case
of a complication.
• Adequate finance and
transport should be
arranged beforehand.
• A blood donor should
be identified for any
unforeseen emergencies.
Care and support by husband
and mother-in-law gives
emotional support and confidence
in child bearing...

Retrieved Chunk 4: 5.  Consu

In [3]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10})

query = "How does folic acid help during pregnancy?"
retrieved_chunks = retriever.get_relevant_documents(query)

print(f"Top {len(retrieved_chunks)} retrieved chunks using MMR:\n")
print(retrieved_chunks[0])
for i, chunk in enumerate(retrieved_chunks):
    print(f"Chunk {i+1}: {chunk.page_content[:300]}...\n")

Top 5 retrieved chunks using MMR:

page_content='likely to express her particular needs and wants while planning for the delivery.  
Antenatal Care and Skilled Attendance at Birth by ANMs/LHVs/SNsGUIDELINES
10
/circle6 All women in the reproductive age group should be advised to have folic acid for 2–3
months pre-conception and continue with it during the fi rst 12 weeks of pregnancy.
Th  is remarkably reduces the incidence of neural tube defects in the foetus. A daily
dose of 400 μg folic acid taken orally is the recommended daily dose.
/circle6 Low iodine levels during pregnancy can cause cretinism, which can lead to mental/
physical retardation of the baby. So regular consumption of iodated salts is advised, as
a prophylactic measures.
Detection of pregnancy
Th e simplest way for you to confi  rm pregnancy in the fi rst trimester is to conduct a urine
examination using a pregnancy test kit. Th e pregnancy test should be off ered to any woman
in the reproductive age group who comes t

In [7]:
vector_store

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from typing import List, Dict, Any, Optional, Tuple, Union
class MedicalRAG:
    """RAG system for medical documents with advanced filtering and retrieval."""
    
    def __init__(
        self,
        vector_store: FAISS,
        model_name: str = "gpt-4-turbo",
        temperature: float = 0.0,
    ):
        self.vector_store = vector_store
        self.llm = ChatOpenAI(model_name=model_name, temperature=temperature)
        
        # Create RAG prompt template
        self.rag_prompt = PromptTemplate(
            template="""You are a medical information assistant that helps healthcare professionals by providing evidence-based information from medical guidelines and literature.

Context information from medical documents:
{context}

Question: {question}

Instructions:
1. Answer based only on the provided context. If the information isn't in the context, say "I don't have enough information to answer this question based on the provided medical guidelines."
2. Cite specific recommendations, evidence levels, and document sources when available.
3. Be concise but comprehensive.
4. If multiple recommendations or conflicting guidance exists in the context, present all perspectives.
5. When quoting recommendations, preserve their exact wording.

Answer:""",
            input_variables=["context", "question"]
        )
        
        # Setup retrieval chain
        self.retriever = vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 6}
        )
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": self.rag_prompt}
        )
    
    def query(self, question: str, filters: Dict[str, Any] = None) -> Dict[str, Any]:
        """Query the RAG system with optional metadata filtering."""
        if filters:
            # Apply metadata filters to the retriever
            self.retriever.search_kwargs["filter"] = filters
        
        # Get answer
        result = self.qa_chain({"query": question})
        
        # Format response
        sources = []
        for doc in result.get("source_documents", []):
            # Add source information
            source_info = {
                "content": doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                "metadata": doc.metadata
            }
            sources.append(source_info)
        
        return {
            "question": question,
            "answer": result["result"],
            "sources": sources
        }
    
    def query_recommendations(self, question: str, evidence_level: str = None) -> Dict[str, Any]:
        """Query specifically for medical recommendations with optional evidence filtering."""
        filters = {"chunk_type": "recommendation"}
        
        if evidence_level:
            # Filter by evidence level (e.g., "high", "moderate", "low")
            filters["evidence_level"] = {"$regex": f"{evidence_level.lower()}.*evidence"}
        
        return self.query(question, filters)



In [6]:
rag = MedicalRAG(vector_store)
    
    # # Run query if provided
    # if args.query:
    #     if args.filter_recommendations:
    #         result = rag.query_recommendations(args.query, args.evidence_level)
    #     else:
query="How does folic acid help during pregnancy?"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")


/tmp/ipykernel_3919087/195178156.py:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(model_name=model_name, temperature=temperature)
/tmp/ipykernel_3919087/195178156.py:58: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa_chain({"query": question})



QUESTION:
How does folic acid help during pregnancy?

ANSWER:
Folic acid supplementation during pregnancy is crucial for preventing neural tube defects in the fetus. According to the guidelines provided:

1. **Pre-conception and Early Pregnancy**: It is recommended that all women in the reproductive age group should take folic acid for 2–3 months pre-conception and continue during the first 12 weeks of pregnancy. The recommended daily dose is 400 μg taken orally. This practice remarkably reduces the incidence of neural tube defects in the fetus (Antenatal Care and Skilled Attendance at Birth by ANMs/LHVs/SNsGUIDELINES).

2. **Evidence Supporting Folic Acid Supplementation**: The recommendation to commence folic acid as early as possible, ideally before conception, is based on evidence that it prevents neural tube defects. This recommendation supersedes the previous WHO recommendation found in the 2012 Guideline: daily iron and folic acid supplementation in pregnant women (WHO recommen

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from typing import List, Dict, Any, Optional, Tuple, Union
class MedicalRAG:
    """RAG system for medical documents with advanced filtering and retrieval."""
    
    def __init__(
        self,
        vector_store: FAISS,
        model_name: str = "gpt-4-turbo",
        temperature: float = 0.0,
    ):
        self.vector_store = vector_store
        self.llm = ChatOpenAI(model_name=model_name, temperature=temperature)
        
        # Create RAG prompt template
        self.rag_prompt = PromptTemplate(
            template="""You are a medical information assistant that helps healthcare professionals by providing evidence-based information from medical guidelines and literature.

Context information from medical documents:
{context}

Question: {question}

Instructions:
1. Answer based only on the provided context. If the information isn't in the context, say "I don't have enough information to answer this question based on the provided medical guidelines."
2. Cite specific recommendations, evidence levels, and document sources when available.
3. Be concise but comprehensive.
4. If multiple recommendations or conflicting guidance exists in the context, present all perspectives.
5. When quoting recommendations, preserve their exact wording.

Answer:""",
            input_variables=["context", "question"]
        )
        
        # Setup retrieval chain
        self.retriever = vector_store.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 6}
        )
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": self.rag_prompt}
        )
    
    def query(self, question: str, filters: Dict[str, Any] = None) -> Dict[str, Any]:
        """Query the RAG system with optional metadata filtering."""
        if filters:
            # Apply metadata filters to the retriever
            self.retriever.search_kwargs["filter"] = filters
        
        # Get answer
        result = self.qa_chain({"query": question})
        
        # Format response
        sources = []
        for doc in result.get("source_documents", []):
            # Add source information
            source_info = {
                "content": doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                "metadata": doc.metadata
            }
            sources.append(source_info)
        
        return {
            "question": question,
            "answer": result["result"],
            "sources": sources
        }
    
    def query_recommendations(self, question: str, evidence_level: str = None) -> Dict[str, Any]:
        """Query specifically for medical recommendations with optional evidence filtering."""
        filters = {"chunk_type": "recommendation"}
        
        if evidence_level:
            # Filter by evidence level (e.g., "high", "moderate", "low")
            filters["evidence_level"] = {"$regex": f"{evidence_level.lower()}.*evidence"}
        
        return self.query(question, filters)



In [8]:
rag = MedicalRAG(vector_store)
    
    # # Run query if provided
    # if args.query:
    #     if args.filter_recommendations:
    #         result = rag.query_recommendations(args.query, args.evidence_level)
    #     else:
query="How does folic acid help during pregnancy?"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")



QUESTION:
How does folic acid help during pregnancy?

ANSWER:
Folic acid supplementation during pregnancy is recommended to significantly reduce the incidence of neural tube defects in the fetus. According to the guidelines, a daily dose of 400 μg of folic acid taken orally is advised for all women in the reproductive age group for 2–3 months pre-conception and during the first 12 weeks of pregnancy (Antenatal Care and Skilled Attendance at Birth by ANMs/LHVs/SNsGUIDELINES). This recommendation is supported by the consensus statement from the Pregnancy Working Group, Geneva, 2015, which underscores the importance of folic acid supplementation during pregnancy to prevent neural tube defects in the developing fetus (Pregnancy Working Group. Consensus statement on folic acid supplementation during pregnancy. Geneva; 2015).

SOURCES:

1. 
   Section: Counselling
   Type: text

2. 
   Section: 18 Screening for Syphilis
   Type: text

3. antenatal care 
for a positive pregnancy experience



In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from typing import List, Dict, Any, Optional, Tuple, Union
class MedicalRAG:
    """RAG system for medical documents with advanced filtering and retrieval."""
    
    def __init__(
        self,
        vector_store: FAISS,
        model_name: str = "gpt-4-turbo",
        temperature: float = 0.0,
    ):
        self.vector_store = vector_store
        self.llm = ChatOpenAI(model_name=model_name, temperature=temperature)
        
        # Create RAG prompt template
        self.rag_prompt = PromptTemplate(
            template=""" You are a medical AI assistant. Answer the medical question based only on the following context.
        If you don't know the answer based on the context, admit that you don't know rather than making up information.
        Always maintain patient confidentiality and provide evidence-based answers when possible.
        
        Context:
        {context}
        
        Medical Question:
        {question}
        
        Answer:
        """,
            input_variables=["context", "question"]
        )
        
        # Setup retrieval chain
        self.retriever = vector_store.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 6}
        )
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": self.rag_prompt}
        )
    
    def query(self, question: str, filters: Dict[str, Any] = None) -> Dict[str, Any]:
        """Query the RAG system with optional metadata filtering."""
        if filters:
            # Apply metadata filters to the retriever
            self.retriever.search_kwargs["filter"] = filters
        
        # Get answer
        result = self.qa_chain({"query": question})
        
        # Format response
        sources = []
        for doc in result.get("source_documents", []):
            # Add source information
            source_info = {
                "content": doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                "metadata": doc.metadata
            }
            sources.append(source_info)
        
        return {
            "question": question,
            "answer": result["result"],
            "sources": sources
        }
    
    def query_recommendations(self, question: str, evidence_level: str = None) -> Dict[str, Any]:
        """Query specifically for medical recommendations with optional evidence filtering."""
        filters = {"chunk_type": "recommendation"}
        
        if evidence_level:
            # Filter by evidence level (e.g., "high", "moderate", "low")
            filters["evidence_level"] = {"$regex": f"{evidence_level.lower()}.*evidence"}
        
        return self.query(question, filters)



In [10]:
rag = MedicalRAG(vector_store)
    
    # # Run query if provided
    # if args.query:
    #     if args.filter_recommendations:
    #         result = rag.query_recommendations(args.query, args.evidence_level)
    #     else:
query="How does folic acid help during pregnancy?"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")



QUESTION:
How does folic acid help during pregnancy?

ANSWER:
Folic acid is crucial during pregnancy as it significantly reduces the incidence of neural tube defects in the fetus. Neural tube defects are serious birth defects of the brain and spine, such as spina bifida and anencephaly. The recommended daily dose of folic acid for women in the reproductive age group is 400 μg taken orally. It is advised to start taking folic acid for 2–3 months pre-conception and continue during the first 12 weeks of pregnancy to achieve its protective effects against neural tube defects.

SOURCES:

1. 
   Section: Counselling
   Type: text

2. 
   Section: 18 Screening for Syphilis
   Type: text

3. antenatal care 
for a positive pregnancy experience

WHO Library Cataloguing-in-Publication Data
WHO recommendations on antenatal care for a positive pregnancy experience
   Section: 9 Blood and Nutrition. In: British > Roll Back Malaria Partnership Malaria in
   Type: text

4. antenatal care 
for a posit

In [11]:
rag = MedicalRAG(vector_store)
    
    # # Run query if provided
    # if args.query:
    #     if args.filter_recommendations:
    #         result = rag.query_recommendations(args.query, args.evidence_level)
    #     else:
query="2.5 months hogae h pregnancy ko but vomet ya kch ni hora mjhe bss upt positive h"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")



QUESTION:
2.5 months hogae h pregnancy ko but vomet ya kch ni hora mjhe bss upt positive h

ANSWER:
If you have a positive pregnancy test but are not experiencing typical pregnancy symptoms like nausea or vomiting, it is still likely that you are pregnant. Some women do not experience these symptoms, especially early in pregnancy. Each pregnancy is unique, and the presence or absence of certain symptoms does not necessarily indicate a problem.

However, it is important to have regular antenatal check-ups to monitor the health of both you and your baby. Since you are around 2.5 months pregnant, you should schedule an appointment with your healthcare provider if you haven't already done so. During this visit, your healthcare provider can confirm the pregnancy through a clinical assessment and possibly an ultrasound, discuss what to expect in the coming months, and address any questions or concerns you might have.

Remember, regular antenatal care is crucial for ensuring a healthy pregna

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from typing import List, Dict, Any, Optional, Tuple, Union
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.llms import HuggingFacePipeline
class MedicalRAG:
    """RAG system for medical documents with advanced filtering and retrieval."""
    
    def __init__(
        self,
        vector_store: FAISS,
        model_name: str =  "/data/models/huggingface/meta-llama/Llama-3.2-3B-Instruct",
        temperature: float = 0.2,
        model=model,
        tokenizer=tokenizer

    ):
        self.vector_store = vector_store
        
        
        # Create text generation pipeline
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=512,
            temperature=temperature,  # Lower temperature for medical accuracy
            top_p=0.95
        )
        self.device = "cuda"
        # Create LangChain wrapper
        self.llm = HuggingFacePipeline(pipeline=pipe)
        
        # Create RAG prompt template
        self.template=""" You are a medical AI assistant. Answer the medical question based only on the following context.
        If you don't know the answer based on the context, admit that you don't know rather than making up information.
        Always maintain patient confidentiality and provide evidence-based answers when possible.
        
        Context:
        {context}
        
        Medical Question:
        {question}
        
        Answer:
        
        """
        self.rag_prompt = PromptTemplate(
            template=self.template,
            input_variables=["context", "question"]
        )
        
        # Setup retrieval chain
        self.retriever = vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 6}
        )
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": self.rag_prompt}
        )
    
    def query(self, question: str, filters: Dict[str, Any] = None) -> Dict[str, Any]:
        """Query the RAG system with optional metadata filtering."""
        if filters:
            # Apply metadata filters to the retriever
            self.retriever.search_kwargs["filter"] = filters
        
        # Get answer
        result = self.qa_chain({"query": question})
        answer = result["result"]

    # Find "Answer:" and return everything after it
        answer_start = answer.lower().find("answer:")
        if answer_start != -1:
            answer = answer[answer_start:].strip()
        # Format response
        sources = []
        for doc in result.get("source_documents", []):
            # Add source information
            source_info = {
                "content": doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                "metadata": doc.metadata
            }
            sources.append(source_info)
        start=len(result["result"])
        return {
            "question": question,
            "answer": answer,
            "sources": sources
        }
    
    def query_recommendations(self, question: str, evidence_level: str = None) -> Dict[str, Any]:
        """Query specifically for medical recommendations with optional evidence filtering."""
        filters = {"chunk_type": "recommendation"}
        
        if evidence_level:
            # Filter by evidence level (e.g., "high", "moderate", "low")
            filters["evidence_level"] = {"$regex": f"{evidence_level.lower()}.*evidence"}
        
        return self.query(question, filters)



In [1]:
rag = MedicalRAG(vector_store,model=model, tokenizer=tokenizer)
    
    # # Run query if provided
    # if args.query:
    #     if args.filter_recommendations:
    #         result = rag.query_recommendations(args.query, args.evidence_level)
    #     else:
query="जयदा टाइम तक बैठने से चक्कर आना खड़ा होने से पेट दर्द होने लगता है"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")


NameError: name 'MedicalRAG' is not defined

In [3]:
model_name =  "/data/models/huggingface/meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
            model_name
        ).to("cuda")

Loading checkpoint shards: 100%|██████████| 2/2 [01:12<00:00, 36.01s/it]


In [12]:
rag = MedicalRAG(vector_store)
    
    # # Run query if provided
    # if args.query:
    #     if args.filter_recommendations:
    #         result = rag.query_recommendations(args.query, args.evidence_level)
    #     else:
query="How does folic acid help during pregnancy?"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



QUESTION:
How does folic acid help during pregnancy?

ANSWER:
Answer:
        
         Folic acid plays an essential role in preventing congenital anomalies such as spina bifida and anencephaly. It also helps reduce the risk of miscarriage and stillbirth. The recommended daily intake of folic acid during pregnancy is 400 μg, which can be obtained through oral supplements or fortified foods. Taking folic acid before conception and throughout the first 12 weeks of pregnancy significantly reduces the incidence of neural tube defects in the fetus. Therefore, all women of childbearing age should be advised to consume folic acid for 2-3 months prior to conception and continue taking it during the first 12 weeks of pregnancy.

SOURCES:

1. 
   Section: Counselling
   Type: text

2. 
   Section: 18 Screening for Syphilis
   Type: text

3. antenatal care 
for a positive pregnancy experience

WHO Library Cataloguing-in-Publication Data
WHO recommendations on antenatal care for a positive pregn

In [13]:
rag = MedicalRAG(vector_store,model=model, tokenizer=tokenizer)
    
    # # Run query if provided
    # if args.query:
    #     if args.filter_recommendations:
    #         result = rag.query_recommendations(args.query, args.evidence_level)
    #     else:
query="What is the remedy for nausea or vomiting in 1 month pregnancy?"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



QUESTION:
What is the remedy for nausea or vomiting in 1 month pregnancy?

ANSWER:
Answer:
        
         Based on the provided context, there is no specific recommended treatment mentioned for nausea or vomiting during the first month of pregnancy. However, the guidelines suggest that pharmacological treatments like doxylamine and metoclopramide should be reserved for those pregnant women experiencing distressing symptoms that are not relieved by non-pharmacological options, under the supervision of a medical doctor.

The most commonly recommended non-pharmacological option for relieving nausea and vomiting in early pregnancy is ginger, although this is not explicitly mentioned in the provided text. It is essential to note that individual experiences and responses to different remedies can vary greatly, so it's crucial to consult a healthcare provider for personalized guidance.

In general, the American College of Obstetricians and Gynecologists (ACOG) recommends the following non

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_name =  "/data/models/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
            model_name
        ).to("cuda")

/home/vidhij2/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-05 20:52:37.979456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741225957.993906 4005322 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741225957.998167 4005322 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 20:52:38.014567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operatio

: 

In [1]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Total GPU Memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")
print("Memory Allocated:", torch.cuda.memory_allocated() / 1e9, "GB")
print("Memory Reserved:", torch.cuda.memory_reserved() / 1e9, "GB")

CUDA Available: True
Total GPU Memory: 50.896961536 GB
Memory Allocated: 0.0 GB
Memory Reserved: 0.0 GB


In [1]:
!nvidia-smi

Wed Mar  5 20:41:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    On  |   00000000:01:00.0 Off |                  Off |
| 30%   36C    P8             26W /  270W |       2MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install --upgrade transformers

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.0 MB 3.2 MB/s eta 0:00:01|██████                          | 1.9 MB 3.2 MB/s eta 0:00:03
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
  distutils: /home/vidhij2/.local/lib/python3.9/site-packages
  sysconfig: /home/vidhij2/.local/lib64/python3.9/site-packages
  user = True
  home = None
  root = None
  prefix = None


In [1]:
pip install bitsandbytes accelerate transformers torch

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 76.1 MB 33.2 MB/s eta 0:00:01
  distutils: /home/vidhij2/.local/lib/python3.9/site-packages
  sysconfig: /home/vidhij2/.local/lib64/python3.9/site-packages
  user = True
  home = None
  root = None
  prefix = None
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "/data/models/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1"

# Enable 8-bit quantization to reduce VRAM usage
bnb_config = BitsAndBytesConfig(load_in_8bit=True)  # Use load_in_4bit=True if needed

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model in optimized mode
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Mixtral loaded successfully in 8-bit mode.")


/home/vidhij2/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-05 20:56:32.504016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741226192.518052 4009762 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741226192.522254 4009762 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 20:56:32.538413: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operatio

✅ Mixtral loaded successfully in 8-bit mode.


In [7]:
rag = MedicalRAG(vector_store,model=model, tokenizer=tokenizer)
    
query="2.5 months hogae h pregnancy ko but vomet ya kch ni hora mjhe bss upt positive h"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")


Device set to use cuda:0
/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



QUESTION:
2.5 months hogae h pregnancy ko but vomet ya kch ni hora mjhe bss upt positive h

ANSWER:
Answer:
        
        2.5 months ho gai hogae h pregnancy ko par vomiting ya kuch nahi hota hai, lekin agar
        vomiting ho raha hai to doctor se consult kar lijiye. Urine mein Human Chorionic
        Gonadotrophin (hCG) hormone detect ki ja sakti hai jo pratyek maheene ke doosre din se shuru
        hota hai aur apni madad kar sakta hai pregnancy test kit ka istemaal kar pregnancy confirm
        karne ke liye.

SOURCES:

1. 
   Section: Counselling
   Type: text

2. 
   Section: These cards are designed for individual counselling but may be used for group counselling if it is possible to form a homogenous group of
   Type: text

3. 
   Section: 8 Schedule for MMA Trainings > MMA is a safe and effective method to terminate early
   Type: text

4. 
   Section: The presence of meconium
   Type: text

5. 
   Section: Confirm
   Type: text

6. 
   Section: Metronidazole
   Type: tex

In [8]:
query="What is the remedy for nausea or vomiting in 1 month pregnancy?"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")


/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



QUESTION:
What is the remedy for nausea or vomiting in 1 month pregnancy?

ANSWER:
Answer:
        
        
        Based on the provided context, for the relief of nausea in early pregnancy, non-pharmacological options are recommended. These options include eating three main meals and one nutritious snack, including calcium-rich foods like milk, curd, paneer, ragi, sesame seeds, bathua leaves, methi leaves, etc., and continuing to consume one Folic Acid tablet (400 mg) daily and dietary sources of folate such as catla fish, moth beans, kidney beans, soybean, spinach, sem ki phali, sarson ka saag, chicken liver. Lemon oil, mint oil, and chamomile have low-certainty evidence suggesting they may make little or no difference in reducing nausea and vomiting symptoms. However, moderate-certainty evidence shows that vitamin B6 (pyridoxine) probably reduces nausea symptoms scores during pregnancy. It is essential to consult with a healthcare provider before starting any new treatment regime

In [9]:
query="How does folic acid help during pregnancy?"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")


/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



QUESTION:
How does folic acid help during pregnancy?

ANSWER:
Answer:
        
        
        Folic acid helps reduce the incidence of neural tube defects in the fetus when taken before conception and during the first 12 weeks of pregnancy. The recommended daily dose is 400 μg taken orally.

SOURCES:

1. 
   Section: Counselling
   Type: text

2. 
   Section: 18 Screening for Syphilis
   Type: text

3. antenatal care 
for a positive pregnancy experience

WHO Library Cataloguing-in-Publication Data
WHO recommendations on antenatal care for a positive pregnancy experience
   Section: 9 Blood and Nutrition. In: British > Roll Back Malaria Partnership Malaria in
   Type: text

4. antenatal care 
for a positive pregnancy experience

WHO Library Cataloguing-in-Publication Data
WHO recommendations on antenatal care for a positive pregnancy experience
   Section: 3 Evidence and recommendations
   Type: text

5. 
   Section: 
   Type: text

6. antenatal care 
for a positive pregnancy experie

In [10]:
query="what should be my diet and what precautions should I keep?"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")


/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



QUESTION:
what should be my diet and what precautions should I keep?

ANSWER:
Answer:
        
        
        Based on the provided context, here are some recommendations for your diet and precautions you should keep:

        Diet:
        - Your diet should consist of a balanced plate where half of it is filled with various vegetables like beans, tomatoes, celery, cabbage, mushrooms, etc. These vegetables provide fiber which helps in controlling postprandial glucose level.
        - Another quarter of your plate should contain protein-rich food such as dal, soy nuggets, tofu, eggs, paneer, chicken, or fish.
        - The last quarter of your plate should include carbohydrates like chapati, brown rice, bread, or cereals.
        - Ensure you have at least one serving of low-fat, sugar-free yogurt, curd, or milk daily.
        - Include at least one serving of fruits like guava, apple, berries, or any citrus fruits in your diet.
        - Try to limit your carbohydrate servings in l

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_name =  "/data/models/huggingface/qwen/Qwen1.5-4B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
            model_name
        ).to("cuda")

/home/vidhij2/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-06 01:06:52.572425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741241212.587841 4088178 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741241212.592333 4088178 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 01:06:52.609327: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operatio

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_name =  "/data/models/huggingface/tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
            model_name
        ).to("cuda")

/home/vidhij2/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-06 00:04:29.005186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741237469.028514 4077900 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741237469.035355 4077900 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 00:04:29.062210: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operatio

In [18]:

tokenizer = AutoTokenizer.from_pretrained("medalpaca/medalpaca-7b", legacy=True)
model = AutoModelForCausalLM.from_pretrained(
            "medalpaca/medalpaca-7b"
        ).to("cuda")

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


OSError: [Errno 122] Disk quota exceeded

In [14]:
pip install --upgrade transformers sentencepiece tiktoken

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.3 MB 4.1 MB/s eta 0:00:01
  distutils: /home/vidhij2/.local/lib/python3.9/site-packages
  sysconfig: /home/vidhij2/.local/lib64/python3.9/site-packages
  user = True
  home = None
  root = None
  prefix = None
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/vidhij2/.local/lib/python3.9/site-packages
sysconfig: /home/vidhij2/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install transformers torch accelerate

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/vidhij2/.local/lib/python3.9/site-packages
sysconfig: /home/vidhij2/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.


In [14]:
rag = MedicalRAG(vector_store,model=model, tokenizer=tokenizer)
    
query="2.5 months hogae h pregnancy ko but vomet ya kch ni hora mjhe bss upt positive h"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")


Device set to use cuda:0



QUESTION:
2.5 months hogae h pregnancy ko but vomet ya kch ni hora mjhe bss upt positive h

ANSWER:
Answer:

SOURCES:

1. 
   Section: Counselling
   Type: text

2. 
   Section: These cards are designed for individual counselling but may be used for group counselling if it is possible to form a homogenous group of
   Type: text

3. 
   Section: Confirm
   Type: text

4. 
   Section: Confirm
   Type: text

5. 
   Section: Metronidazole
   Type: text

6. 
   Section: These cards are designed for individual counselling but may be used for group counselling if it is possible to form a homogenous group of
   Type: text


In [15]:
query="How does folic acid help during pregnancy?"
result = rag.query(query)
        
print("\n" + "="*50)
print("QUESTION:")
print(result["question"])
print("\nANSWER:")
print(result["answer"])
print("\nSOURCES:")
for i, source in enumerate(result["sources"]):
    print(f"\n{i+1}. {source['metadata'].get('document_title', 'Unknown document')}")
    if "section_path" in source["metadata"]:
        print(f"   Section: {' > '.join(source['metadata']['section_path'])}")
    if "chunk_type" in source["metadata"]:
        print(f"   Type: {source['metadata']['chunk_type']}")
    if "evidence_level" in source["metadata"]:
        print(f"   Evidence level: {source['metadata']['evidence_level']}")



QUESTION:
How does folic acid help during pregnancy?

ANSWER:
Answer:

SOURCES:

1. 
   Section: Counselling
   Type: text

2. antenatal care 
for a positive pregnancy experience

WHO Library Cataloguing-in-Publication Data
WHO recommendations on antenatal care for a positive pregnancy experience
   Section: 3 Evidence and recommendations
   Type: text

3. 
   Section: These cards are designed for individual counselling but may be used for group counselling if it is possible to form a homogenous group of
   Type: text

4. antenatal care 
for a positive pregnancy experience

WHO Library Cataloguing-in-Publication Data
WHO recommendations on antenatal care for a positive pregnancy experience
   Section: 9 Blood and Nutrition. In: British > Roll Back Malaria Partnership Malaria in
   Type: text

5. antenatal care 
for a positive pregnancy experience

WHO Library Cataloguing-in-Publication Data
WHO recommendations on antenatal care for a positive pregnancy experience
   Section: 4 Impleme